In [1]:
from constants import *

print(MODEL_ID)

In [1]:
import json
import random 
import os
from mistralai.client import MistralClient
from dotenv import load_dotenv

def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield json.loads(line)

def extract_content(json_obj):
    messages = []
    for edit in json_obj.get('edits', []):
        content_msg = []
        # Content 1 
        content1 = edit.get('src', {}).get('text')
        content1_path = edit.get('src', {}).get('path')
        content1_lang = edit.get('src', {}).get('lang') 

        # Content 2 
        content2 = edit.get('tgt', {}).get('text')
        content2_path = edit.get('tgt', {}).get('path')
        content2_lang = edit.get('tgt', {}).get('lang')
        
        if content1:
            # \nContent language: {content1_lang} \
            content1_format = f"""The following CODE SNIPPET contains a typo. Please identify the mistake but do not change the original structure:
                                \n\nContent path: "{content1_path}"
                                \nINVALID CODE SNIPPET:\n"{content1.strip()}" """
                                
            content_msg.append({'content': content1_format, 'role': "user"})
        if content2:
            # \nContent language: {content2_lang} \
            content2_format = f"""The typo identified in the code snippet is: "{content2.strip()}"
                                \n\nContent path: "{content2_path}"
                                \nCORRECTED CODE SNIPPET:\n"{content2.strip()}" """

            content_msg.append({'content': content2_format, 'role': "assistant", })

            messages.append({"messages": content_msg})
    return messages

# Example usage
file_path = 'data/typo.jsonl'
all_messages = []

for json_obj in read_jsonl(file_path):
    messages = extract_content(json_obj)
    all_messages.extend(messages)


In [2]:
all_messages = all_messages[:10000]

def write_jsonl(data, file_path):
    ndjson_data = "\n".join(json.dumps(item) for item in data)
    with open(file_path, "w") as jsonl_file:
        jsonl_file.write(ndjson_data)

# Write the messages to a JSONL file
messages_file_path = 'data/messages.jsonl'
write_jsonl(all_messages, messages_file_path)

In [3]:
data = all_messages
data = data[:10000]

# Split data
train_ratio = 0.901
eval_ratio = 0.04 
test_ratio = 0.04

train_size = int(len(data) * train_ratio)
eval_size = int(len(data) * eval_ratio)

train_data = data[:train_size]
eval_data = data[train_size:train_size + eval_size]
test_data = data[train_size + eval_size:]

write_jsonl(train_data, 'data/training_file.jsonl')
write_jsonl(eval_data, 'data/evaluation_file.jsonl')
write_jsonl(test_data, 'data/test_file.jsonl')

In [1]:

load_dotenv('conf/secrets.env')
client = MistralClient(api_key=api_key)

with open("data/training_file.jsonl", "rb") as f:
    training_data = client.files.create(file=("training_file.jsonl", f), purpose="fine-tune")    

with open("data/evaluation_file.jsonl", "rb") as f:
    evaluation_file = client.files.create(file=("evaluation_file.jsonl", f), purpose="fine-tune")    
    

NameError: name 'load_dotenv' is not defined

In [5]:
from mistralai.models.jobs import TrainingParameters

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[training_data.id],
    validation_files=[evaluation_file.id],
    hyperparameters=TrainingParameters(
        training_steps=1,
        learning_rate=0.0001,
        )
)
created_jobs

Job(id='f750409c-3cb7-4de0-8ed9-5e57ecd28446', hyperparameters=TrainingParameters(training_steps=1, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1719641600, modified_at=1719641600, training_files=['ce5440d3-f1e4-4c25-960a-40461f7d3d7f'], validation_files=['d50d315e-b2b7-409a-8fbc-07ed0cbee2b7'], object='job', integrations=[])

In [6]:
jobs = client.jobs.list()
print(jobs)

retrieved_jobs = client.jobs.retrieve(created_jobs.id)
print(retrieved_jobs)

data=[Job(id='f750409c-3cb7-4de0-8ed9-5e57ecd28446', hyperparameters=TrainingParameters(training_steps=1, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1719641600, modified_at=1719641600, training_files=['ce5440d3-f1e4-4c25-960a-40461f7d3d7f'], validation_files=['d50d315e-b2b7-409a-8fbc-07ed0cbee2b7'], object='job', integrations=[]), Job(id='89dd3dee-630a-4c66-b6f9-c7eab654efa8', hyperparameters=TrainingParameters(training_steps=1, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='RUNNING', job_type='FT', created_at=1719641554, modified_at=1719641554, training_files=['24022699-8f73-4a52-b486-c664c74112a6'], validation_files=['d0f6d296-96ee-43ac-bb0f-30670b3780cf'], object='job', integrations=[]), Job(id='a3ed164c-5b8f-4df0-aa39-63a4f324fdb6', hyperparameters=TrainingParameters(training_steps=1, learning_rate=0.0001), fine_tuned_model='ft:open-mistral-7b:b25ac4af:20240606:a3ed164c', model='o

In [9]:
retrieved_job = client.jobs.retrieve(job_id="f750409c-3cb7-4de0-8ed9-5e57ecd28446")

In [10]:
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model=retrieved_job.fine_tuned_model,
    messages=[ChatMessage(role='user', content=f"""The following CODE SNIPPET contains a typo. Please identify the mistake but do not change the original structure:
                                \n\nContent path: "javascripts/db.js"
                                \nINVALID CODE SNIPPET:\n"// Return a promise that either resolves with the most recent doducment" """
                                )]
)



In [11]:
print(chat_response.choices[0].message.content)

The typo in the code snippet is in the word "doducment". It should be "document". Here's the corrected version:

"// Return a promise that either resolves with the most recent document"


### LLM Evaluation method   

In [75]:
# Evaluation 
test_evaluation_dataset = test_data[:10]

In [76]:
test_evaluation_dataset[:1]

[{'messages': [{'content': 'The following CODE SNIPPET contains a typo. Please identify the mistake but do not change the original structure:\n                                \n\nContent path: "javascripts/db.js"\n                                \nINVALID CODE SNIPPET:\n"// Return a promise that either resolves with the most recent doducment" ',
    'role': 'user'},
   {'content': 'The typo identified in the code snippet is: "// Return a promise that either resolves with the most recent docucment"\n                                \n\nContent path: "javascripts/db.js"\n                                \nCORRECTED CODE SNIPPET:\n"// Return a promise that either resolves with the most recent docucment" ',
    'role': 'assistant'}]}]

In [77]:
eval_data[:1]

[{'messages': [{'content': 'The following CODE SNIPPET contains a typo. Please identify the mistake but do not change the original structure:\n                                \n\nContent path: "doc/models.md"\n                                \nINVALID CODE SNIPPET:\n"`C(\'name\')` - a pipeline config option" ',
    'role': 'user'},
   {'content': 'The typo identified in the code snippet is: "- `C(\'name\')` - a pipeline config option"\n                                \n\nContent path: "doc/models.md"\n                                \nCORRECTED CODE SNIPPET:\n"- `C(\'name\')` - a pipeline config option" ',
    'role': 'assistant'}]}]

In [94]:
from mistralai.models.chat_completion import ChatMessage

def chat_model(query):
    query_formatted = query
    context = "You are a github expert in identifying mistakes, understanding user request and suggesting best coding practices."
    
    prompt_template = f"""
                    ### CONTEXT ###
                    {context}

                    ### EXAMPLES ### 
                    The following CODE SNIPPET contains a typo. Please identify the mistake but do not change the original structure:
                    \n\nContent path: "doc/models.md"
                    \nINVALID CODE SNIPPET:\n"`C(\'name\')` - a pipeline config option" ',

                    'The typo identified in the code snippet is: "- `C(\'name\')` - a pipeline config option"\n
                    \n\nContent path: "doc/models.md"\n
                    \nCORRECTED CODE SNIPPET:\n"- `C(\'name\')` - a pipeline config option" ',


                    ### TASK ###
                    {query_formatted}

                    'The typo identified in the code snippet is:
                    \n\nContent path:
                    \nCORRECTED CODE SNIPPET:

                    ## INSTRUCTIONS ## 
                    Return the answer in json format. 
                    """
    
    chat_response = client.chat(
        model=retrieved_job.fine_tuned_model,
        messages=[ChatMessage(role='user', 
                              content=prompt_template
                                )])
    
    return chat_response, chat_response.choices[0].message.content


def chat_model_evaluator(llm1_query, llm1_answer, gold_truth):

    llm1_context = "You are a github expert in identifying mistakes, understanding user request and suggesting best coding practices."
    
    llm_evaluator_task = f"""
                        ### TASK ###
                        Evaluate the response from LLM1. Based on LLM1 CONTEXT, LLM1 TASK and LLM1 RESPONSE TO TASK in comparison to GOLD ANSWER,
                        determine if LLM1 RESPONSE TO TASK is accurate and coherent. 

                        Rate the LLM1 RESPONSE TO TASK on how accurate and coherent it is, return a value between 0 and 1 in JSON FORMAT.
                        Reference EXAMPLE RESPONSE to RATE THE ANSWER ACCURATELY AND NOT JUST 1.0. So responses can be 0.79 or 0.41...
                        {{"accurate_response": String}}
                        """

    prompt_template = f"""
                    ### LLM1 CONTEXT ###
                    {llm1_context}

                    ### LLM1 EXAMPLES ### 
                    The following CODE SNIPPET contains a typo. Please identify the mistake but do not change the original structure:
                    \n\nContent path: "doc/models.md"
                    \nINVALID CODE SNIPPET:\n"`C(\'name\')` - a pipeline config option" ',

                    'The typo identified in the code snippet is: "- `C(\'name\')` - a pipeline config option"\n
                    \n\nContent path: "doc/models.md"\n
                    \nCORRECTED CODE SNIPPET:\n"- `C(\'name\')` - a pipeline config option" ',

                    ### LLM1 TASK ###
                    {llm1_query}
                    

                    ### LLM1 RESPONSE TO TASK ###
                    {llm1_answer}

                    ### GOLD ANSWER ###
                    {gold_truth}

                    ### EXAMPLE RESPONSE ###

                    The following CODE SNIPPET contains a typo. Please identify the mistake but do not change the original structure:
                    \n\nContent path: "doc/models.md"
                    \nINVALID CODE SNIPPET:\n"`C(\'name\')` - a pipeline config option" ',

                    ## COHERENT AND ACCURATE ANSWER ##
                    The mistake in the code snippet is subtle but important. The backticks () are used for indicating code blocks or inline code in Markdown. However, in this case, the backticks are misplaced and should be surrounding the text 'C(\'name\')' instead of single quotes. Here's the corrected version: 
                    \nContent path: "doc/models.md"
                    \nCORRECTED CODE SNIPPET: "C('name')` - a pipeline config option"

                    accurate_response value would be between (0.7 to 1],

                    ## SEMI-COHERENT AND SEMI-ACCURATE ANSWER##
                    There seems to be an issue with the code snippet. The single quotes ('') are used for enclosing text or string values, but in this case, they seem to be misplaced. It might be better to use backticks (`) to indicate a code block or inline code. Here's a suggested correction:
                    \nContent path: "doc/models.md"
                    \nCORRECTED CODE SNIPPET: "C('name') - a pipeline config option"

                    accurate_response value would be between [0.4 to 0.7)

                    ## NON-COHERENT AND NON-ACCURATE ANSWER, accurate_response is rated 0.26 ##
                    The code snippet looks a bit off. I think the single quotes (') are causing the issue. Maybe we should try using double quotes ("") instead? Here's a possible fix:
                    \nContent path: "doc/models.md"
                    \nCORRECTED CODE SNIPPET: ""C('name')"" - a pipeline config option"

                    accurate_response value would be between [0 to 0.4)


                    ### TASK ###
                    {llm_evaluator_task}

                    """
    
    model = "open-mistral-7b"
    chat_response = client.chat(
        model=model,
        messages=[ChatMessage(role='user', 
                              content=prompt_template
                                )])
    
    return chat_response, chat_response.choices[0].message.content




In [95]:
from langchain_mistralai.chat_models import ChatMistralAI

In [96]:
full_responses = []
only_chat_responses = []
full_response_evaluators = []
only_chat_response_evaluators = []
gold_truths = []
for test_jsonl_dict in test_evaluation_dataset:
    incorrect_test_content_query = test_jsonl_dict['messages'][0]['content']
    correct_test_content_query = test_jsonl_dict['messages'][1]['content']

    full_response, only_chat_response = chat_model(query=incorrect_test_content_query)
    
    full_response_evaluator, only_chat_response_evaluator = chat_model_evaluator(llm1_query=incorrect_test_content_query
                                                                                 , llm1_answer=only_chat_response
                                                                                 , gold_truth = correct_test_content_query)
    full_responses.append(full_response)
    gold_truths.append(correct_test_content_query)
    only_chat_responses.append(only_chat_response)
    full_response_evaluators.append(full_response_evaluator)
    only_chat_response_evaluators.append(only_chat_response_evaluator)

In [97]:
import pandas as pd 
response_evaluation_dataframe = pd.DataFrame({
    "llm1_full_response":full_responses,
    "llm1_extracted_answers":only_chat_responses,
    "gold_truth": gold_truths,
    "evaluator_full_response": full_response_evaluators,
    "evaluator_extracted_answers" :only_chat_response_evaluator
    
})

In [98]:
only_chat_response_evaluator

'{"accurate_response": 0.82}\n\nThe LLM1 response to the task is accurate and coherent. The typo identified is the missing backtick before the text \'C(\'name\')\'. The corrected code snippet correctly uses the backtick to indicate a code block or inline code. The response is also well-structured, with clear identification of the typo and the corrected code snippet. Overall, the response accurately identifies the typo and provides a correct correction, earning it a high accuracy score.'

In [99]:
import json

def extract_accurate_response(input_string):
    try:
        # Extract the JSON part of the string
        json_part = input_string.split('\n\n')[0]
        # Parse the JSON string
        data = json.loads(json_part)
        # Extract and return the accurate response
        return data.get("accurate_response")
    except (json.JSONDecodeError, IndexError):
        return None

response_evaluation_dataframe['accurate_class'] = response_evaluation_dataframe['evaluator_extracted_answers'].apply(lambda x: extract_accurate_response(x))
response_evaluation_dataframe


,llm1_full_response,llm1_extracted_answers,gold_truth,evaluator_full_response,evaluator_extracted_answers,accurate_class
0,id='0e8ade0af62a4e4c8ced2d1bbafc5c93' object='...,"{\n ""content_path"": ""javascripts/db.js"",\n ""...","The typo identified in the code snippet is: ""/...",id='f0b2c71d7fdf4051881ea9a8c6ee95eb' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
1,id='82dd9dfca4c245ddbabe44b34b071496' object='...,"{\n ""content_path"": ""javascripts/db.js"",\n ""...","The typo identified in the code snippet is: ""/...",id='6081c40ce6b7465a95600885484d183f' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
2,id='855f1e20c3d249c29cbc0255d07fd320' object='...,"{\n ""content_path"": ""routes/pop.js"",\n ""typo...","The typo identified in the code snippet is: ""/...",id='db0327709dc54dfebd1aaae8b9aaf20b' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
3,id='1ea8f3e32157462fbdf5118a55cc5a22' object='...,"{\n ""Content path"": ""README.md"",\n ""Invalid ...","The typo identified in the code snippet is: ""A...",id='03a922002cd14ba7a615096f0297d9e5' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
4,id='7248a54b28104f53890fd673452b024a' object='...,"{\n ""content_path"": ""contents.json"",\n ""typo...","The typo identified in the code snippet is: """"...",id='6b6c543f1b9a4c4bb5053183bf7a138e' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
5,id='6e9be35487ac4360811e0b38f51a0b68' object='...,"{\n ""content_path"": ""README.rst"",\n ""typo_id...","The typo identified in the code snippet is: ""L...",id='26bdad95e9fc45cd921cc9e5faa41256' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
6,id='a9b2124470584d1bb5a0429499dbb000' object='...,"{\n ""Content path"": ""README.rst"",\n ""Typo id...","The typo identified in the code snippet is: ""J...",id='8fdd13acc23746938e3a3f9d91ae826e' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
7,id='75f49e7a6a99432c8d8b9da9105c4533' object='...,"{\n ""Content path"": ""README.rst"",\n ""Invalid...","The typo identified in the code snippet is: ""-...",id='505689a378004f27973d1f7309815184' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
8,id='75150175a54f4db2a42cea3d9e6fc273' object='...,"{\n ""content_path"": ""FileProvider.podspec"",\n...","The typo identified in the code snippet is: ""s...",id='e0abcda2b5fc49c79e2cbf654f140052' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82
9,id='20285af092f74ddda7547cf2e7100738' object='...,"{\n ""content_path"": ""documentation/index.jade...","The typo identified in the code snippet is: ""p...",id='9b5914092a064faaaae330b733c481a7' object='...,"{""accurate_response"": 0.82}\n\nThe LLM1 respon...",0.82


In [100]:
response_evaluation_dataframe['accurate_class'] = response_evaluation_dataframe['accurate_class'].astype('float')

In [101]:
print("Model Accuracy of Mistral Ability to reply user based on coherent and accurate changes is", sum(response_evaluation_dataframe['accurate_class'])/len(response_evaluation_dataframe['accurate_class']))

Model Accuracy of Mistral Ability to reply user based on coherent and accurate changes is 0.82


### Conclusion & Learning Points 



#### Fine Tunning


#### Evaluation 
- Model evaluation if not done well at the start is expensive and difficult to reevaluate. 
- Model ability to generate response that is coherent with semantic meaning can be evaluated faster by another LLM rather than with human evaluation if time is of an essence. The ability for the model to improve this evaluation process can be better evaluated if we provide a few shot example of what constitute as coherent , semi coherent and not coherent. - but the ability for the model to generate a range of values without a marking criteria is not possible. 
-  more improvements can be made to the prompt to break down the problem like a math issue; providing a rubric for marking along with COT prompting. 
